In [141]:
import numpy as np
import cv2

res_fac = 1

img = cv2.imread('img/raw/2.png')
h, w, _ = img.shape

img_resized = cv2.resize(img, (w // res_fac, h // res_fac))
blurred = cv2.blur(img_resized, (7, 7))
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

mask = cv2.inRange(hsv, (37, 75, 0), (255, 255, 255))
mask = 255 - mask

kernel = np.ones((5, 5), np.uint8)
morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

connectivity = 4
output = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)

num_labels = output[0]
labels = output[1]
stats = output[2]

filtered = np.zeros_like(morph)

for i in range(1, num_labels):
    a = stats[i, cv2.CC_STAT_AREA]
    top = stats[i, cv2.CC_STAT_TOP]
    left = stats[i, cv2.CC_STAT_LEFT]
    width = stats[i, cv2.CC_STAT_WIDTH]
    height = stats[i, cv2.CC_STAT_HEIGHT]
    
    if (a > 1500):
        filtered[np.where(labels == i)] = 255
        
        # cv2.putText(img_resized, str(a), (left, top), cv2.FONT_ITALIC, 1, (0, 0, 255), 2, cv2.LINE_4)
        # cv2.rectangle(img_resized, (left, top), (left + width, top + height), (0, 255, 0), 1)



gray = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT, dp=1.1, minDist=200, param1=100, param2=0.6, minRadius=100)

if (circles is not None):
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img_resized,(int(i[0]), int(i[1])),int(i[2]), (0,255,0), 2)
            # draw the center of the circle
            cv2.circle(img_resized,(int(i[0]), int(i[1])), 2, (0,0,255), 3)
            
# if circles is not None:
#     circles = np.round(circles[0, :]).astype("int")
#     for (x, y, r) in circles:
#         cv2.circle(img_resized, (x, y), r, (0, 255, 0), 2)
        
else:
    print("No circles")


cv2.namedWindow('custom window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom window', img_resized)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.namedWindow('custom mask window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom mask window', mask)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.namedWindow('custom filt mask window', cv2.WINDOW_KEEPRATIO)
cv2.imshow('custom filt mask window', filtered)
cv2.resizeWindow('custom window', w // 2, h // 2)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(10)



No circles


-1

In [72]:
import cv2
import numpy as np

res_fac = 4

cv2.namedWindow('mask')

def nothing(x):
    pass

cv2.createTrackbar('lh', 'mask', 0, 255, nothing)
cv2.createTrackbar('ls', 'mask', 96, 255, nothing)
cv2.createTrackbar('lv', 'mask', 69, 255, nothing)
cv2.createTrackbar('hh', 'mask', 53, 255, nothing)
cv2.createTrackbar('hs', 'mask', 173, 255, nothing)
cv2.createTrackbar('hv', 'mask', 116, 255, nothing)

img = cv2.imread('img/raw/1.png')
h, w, _ = img.shape
img_resized = cv2.resize(img, (w // res_fac, h // res_fac))


hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)

while True:
    lh = cv2.getTrackbarPos('lh', 'mask')
    ls = cv2.getTrackbarPos('ls', 'mask')
    lv = cv2.getTrackbarPos('lv', 'mask')
    hh = cv2.getTrackbarPos('hh', 'mask')
    hs = cv2.getTrackbarPos('hs', 'mask')
    hv = cv2.getTrackbarPos('hv', 'mask')

    mask = cv2.inRange(hsv, (lh, ls, lv), (hh, hs, hv))

    mask = 255 - mask
    
    cv2.imshow('mask', mask)

    connectivity = 4  
    output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]

    filtered = np.zeros_like(mask)

    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        top = stats[i, cv2.CC_STAT_TOP]
        left = stats[i, cv2.CC_STAT_LEFT]
        width = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]
        #print(a)
        
        if (a >= 50):
            filtered[np.where(labels == i)] = 255
            
            cv2.putText(img_resized, str(a), (left, top), cv2.FONT_ITALIC, 1, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle(img_resized, (left, top), (left + width, top + height), (0, 255, 0), 1)


    cv2.imshow('img', img_resized)
    #cv2.imshow('hsv', hsv[:, :, 0])
    cv2.imshow('mask_filtered', filtered)

    key = cv2.waitKey(80) & 0xFF
    
    if (key == ord('q')):
        break
    
    
cv2.destroyAllWindows()
cv2.waitKey(10)

-1